<a href="https://colab.research.google.com/github/raghavkaushik/DeepLearning/blob/main/CNN_DenseNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, BatchNormalization, Input, Dropout, concatenate, Activation, AveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as plt
import os
import math

In [2]:
#training parameters
batch_size=32
epochs=200
data_augmentation=True
growth_rate=12
num_classes=10
num_dense_blocks=3
depth=100
num_bottleneck_layers = (depth - 4) // (2 * num_dense_blocks)
num_filters_bef_dense_block = 2 * growth_rate
compression_factor = 0.5

In [3]:
(Xtrain,Ytrain),(Xtest,Ytest)=cifar10.load_data()
input_shape=Xtrain.shape[1:]
print("The input_shape:",input_shape)

Xtrain,Xtest=Xtrain/255.0, Xtest/255.0
print('x_train shape:', Xtrain.shape)
print(Xtrain.shape[0], 'train samples')
print(Xtest.shape[0], 'test samples')
print('y_train shape:', Ytrain.shape)

The input_shape: (32, 32, 3)
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)


In [4]:
def lr_schedule(epoch):
  #Learning Rate scheduler 
  lr=1e-3
  if epoch > 180:
        lr *= 0.5e-3
  elif epoch > 160:
        lr *= 1e-3
  elif epoch > 120:
        lr *= 1e-2
  elif epoch > 80:
        lr *= 1e-1
  print('Learning rate: ', lr)
  return lr

In [6]:
inputs=Input(shape=input_shape)
x=BatchNormalization()(inputs)
x=Activation('relu')(x)
x=Conv2D(num_filters_bef_dense_block, kernel_size=3, padding='same', kernel_initializer='he_normal')(x)
x=concatenate([inputs,x])

for i in range(num_dense_blocks):
  for j in range(num_bottleneck_layers):
    y=BatchNormalization()(x)
    y=Activation('relu')(y)
    y=Conv2D(4 * growth_rate, kernel_size=1, padding='same', kernel_initializer='he_normal')(y)
    if not data_augmentation:
      y=Dropout(0.2)(y)
    y=BatchNormalization()(y)
    y=Activation('relu')(y)
    y=Conv2D(growth_rate, kernel_size=3, padding='same', kernel_initializer='he_normal')(y)
    if not data_augmentation:
      y=Dropout(0.2)(y)
    x=concatenate([x,y])
    if i==num_dense_blocks-1:
      continue
    num_filters_bef_dense_block += num_bottleneck_layers * growth_rate
    num_filters_bef_dense_block = int(num_filters_bef_dense_block * compression_factor)
    y=BatchNormalization()(x)
    y=Conv2D(num_filters_bef_dense_block, kernel_size=1, padding='same', kernel_initializer='he_normal')(y)
    if not data_augmentation:
      y=Dropout(0.2)(y)
    #x=AveragePooling2D()(y)

x=AveragePooling2D(pool_size=8)(x)
y=Flatten()(x)
outputs = Dense(num_classes,
                kernel_initializer='he_normal',
                activation='softmax')(y)


In [8]:
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=RMSprop(1e-3),
              metrics=['accuracy'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 batch_normalization_19 (BatchN  (None, 32, 32, 3)   12          ['input_2[0][0]']                
 ormalization)                                                                                    
                                                                                                  
 activation_13 (Activation)     (None, 32, 32, 3)    0           ['batch_normalization_19[0][0]'] 
                                                                                                  
 conv2d_19 (Conv2D)             (None, 32, 32, 189)  5292        ['activation_13[0][0]']    

In [9]:
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_densenet_model.{epoch:02d}.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

In [10]:
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

In [11]:
lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

In [12]:
callbacks = [checkpoint, lr_reducer, lr_scheduler]

In [ ]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(Xtrain, Ytrain,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(Xtest, Ytest),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # preprocessing  and realtime data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (deg 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally
        height_shift_range=0.1,  # randomly shift images vertically
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    datagen.fit(Xtrain)

    steps_per_epoch = math.ceil(len(Xtrain) / batch_size)
    # fit the model on the batches generated by datagen.flow().
    model.fit(x=datagen.flow(Xtrain, Ytrain, batch_size=batch_size),
              verbose=1,
              epochs=epochs,
              validation_data=(Xtest, Ytest),
              steps_per_epoch=steps_per_epoch,
              callbacks=callbacks)

Using real-time data augmentation.
Learning rate:  0.001
Epoch 1/200


In [8]:
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])